### portmy database: stocks table
### stock database: buy, price tables

In [2]:
import pandas as pd
import numpy as np
import os
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

today = date.today()
print(today)

2025-08-24


In [4]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-08-24
yesterday: 2025-08-22 00:00:00


In [6]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 4)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2025-08-18
yesterday: 2025-08-22


### Restart & Run All Cells

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Weekly
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


### This process affects only already in port stocks. To highlight price changes in percent.

In [14]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [18]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_buy = pd.read_sql(sql, const)

names = df_buy.name.tolist()
portfolio = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
    SELECT name, price AS price_d 
    FROM price 
    WHERE date = '{}' AND name IN ({})
    ORDER BY name, date
""".format(yesterday, portfolio)

df_today = pd.read_sql(sql, const)
df_today.shape[0]

29

In [20]:
sql = """
    SELECT name, price AS price_w
    FROM price 
    WHERE date = '{}' AND name IN ({}) 
    ORDER BY name
""".format(week_ago, portfolio)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]

29

In [24]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.tail()

,name,price_d,price_w
24,TFFIF,6.05,6.05
25,TOA,14.60,15.10
26,TVO,24.40,25.25
27,WHAIR,5.40,5.40
28,WHART,9.00,9.15


In [26]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [28]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [30]:
trend.tail()

,name,price_d,price_w,percent
24,TFFIF,6.05,6.05,0.00
25,TOA,14.60,15.10,-3.31
26,TVO,24.40,25.25,-3.37
27,WHAIR,5.40,5.40,0.00
28,WHART,9.00,9.15,-1.64


In [32]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [34]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
14,ORI,2.26,2.38,-5.04%,Worse
26,TVO,24.40,25.25,-3.37%,Worse
25,TOA,14.60,15.10,-3.31%,Worse
13,NER,4.30,4.44,-3.15%,Worse
18,RCL,26.75,27.50,-2.73%,Worse
2,AIMIRT,9.55,9.75,-2.05%,Worse
28,WHART,9.00,9.15,-1.64%,Worse
1,AH,14.60,14.80,-1.35%,Worse
12,MCS,8.45,8.55,-1.17%,Worse
6,CPNREIT,11.40,11.50,-0.87%,Worse


In [36]:
file_name = 'trend.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [38]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\trend.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\trend.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\trend.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\trend.csv


In [40]:
trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(god_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(icd_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [46]:
mask = trend.perform == 'Worse'
trend[mask].tail()

,name,price_d,price_w,percent,perform
16,PTT,32.00,32.25,-0.78,Worse
18,RCL,26.75,27.50,-2.73,Worse
25,TOA,14.60,15.10,-3.31,Worse
26,TVO,24.40,25.25,-3.37,Worse
28,WHART,9.00,9.15,-1.64,Worse


In [48]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,41.38%
Worse,41.38%
No Change,17.24%


In [50]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(246, 8)

In [52]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [54]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).tail()

,name,price_d,price_w,percent,perform,max,min,pe,pbv,volume,beta,market,mrkt
15,PTG,7.15,6.95,2.88,Better,16.40,12.90,26.07,2.59,187.88,1.24,SET100 / SETTHSI,SET100
23,SYNEX,11.60,11.20,3.57,Better,31.00,14.50,15.26,3.52,35.16,2.04,sSET / SETTHSI,SET999
9,IVL,22.80,21.10,8.06,Better,52.75,37.00,4.84,1.09,859.32,1.11,SET50 / SETTHSI,SET50
19,SCC,223.00,205.00,8.78,Better,402.00,307.00,13.98,1.04,815.52,0.73,SET50 / SETCLMV / SETHD / SETTHSI,SET50
17,PTTGC,27.00,22.50,20.00,Better,58.75,39.75,999.99,0.77,961.87,1.13,SET50 / SETCLMV / SETTHSI,SET50


In [56]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
16,PTT,32.25,32.00,-0.78%,Worse,SET50
11,JMT,11.40,11.40,0.00%,No Change,SET50
5,CPF,23.80,24.30,2.10%,Better,SET50
10,JMART,8.15,8.35,2.45%,Better,SET50
3,AWC,2.36,2.42,2.54%,Better,SET50
9,IVL,21.10,22.80,8.06%,Better,SET50
19,SCC,205.00,223.00,8.78%,Better,SET50
17,PTTGC,22.50,27.00,20.00%,Better,SET50


In [58]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,75.00%
No Change,12.50%
Worse,12.50%


In [60]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
14,ORI,2.38,2.26,-5.04%,Worse,SET100
18,RCL,27.50,26.75,-2.73%,Worse,SET100
21,SINGER,5.25,5.25,0.00%,No Change,SET100
22,STA,12.00,12.00,0.00%,No Change,SET100
4,BCH,13.20,13.40,1.52%,Better,SET100
15,PTG,6.95,7.15,2.88%,Better,SET100


In [62]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Better,33.33%
Worse,33.33%
No Change,33.33%


In [64]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
26,TVO,25.25,24.40,-3.37%,Worse,SET999
25,TOA,15.10,14.60,-3.31%,Worse,SET999
13,NER,4.44,4.30,-3.15%,Worse,SET999
2,AIMIRT,9.75,9.55,-2.05%,Worse,SET999
28,WHART,9.15,9.00,-1.64%,Worse,SET999
1,AH,14.80,14.60,-1.35%,Worse,SET999
12,MCS,8.55,8.45,-1.17%,Worse,SET999
6,CPNREIT,11.50,11.40,-0.87%,Worse,SET999
8,GVREIT,6.45,6.40,-0.78%,Worse,SET999
24,TFFIF,6.05,6.05,0.00%,No Change,SET999


In [66]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,
Worse,60.00%
Better,26.67%
No Change,13.33%


In [68]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [70]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,proportion
perform,


In [72]:
const.close()
conmy.close()

In [74]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-24 21:12:18
